<a href="https://colab.research.google.com/github/pooyapaydary/Python/blob/main/micro_project_01Nov25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import everything
import pandas as pd
import re
import numpy as np
import sqlite3

In [90]:
#read the data
wdf = pd.read_csv("water_quality_readings.csv")
sdf = pd.read_csv("stations.csv")

#look at the data
#print(wdf.head())
#print("\n")
#print(sdf.head(5))

#print(wdf.info())
#print("\n")
#print(sdf.info())

#cleaning the data
  #1-change sample time to date-time
  #2-trim string with function
     #define function
def clean_up(c):

  if type(c)==str:
      c = c.lower().strip()  #lower and strip spaces
      c= re.sub(r'[^A-Za-z0-9_ ]+', "", c)  #remove weird charchters
      if c =="nan":
        return (None)
  return(c)

#apply the cleaning function to the dataframe
for col  in wdf.columns :
  wdf[col]=wdf[col].apply(clean_up)

for col  in sdf.columns :
  sdf[col]=sdf[col].apply(clean_up)


#change date time to date time
wdf["sample_time"] = pd.to_datetime(wdf["sample_time"], errors="coerce")

#impute missing pH values with station-wise median

#I learned this code here. "transform"
wdf["pH"] = wdf["pH"].fillna(wdf.groupby("station_id")["pH"].transform("median"))

#lets do it for Turbidity
wdf["turbidity_NTU"]=wdf["turbidity_NTU"].fillna(wdf.groupby("station_id")["turbidity_NTU"].transform("median"))


#lets write a function to do that to all columns in wdf
def med (c):
  wdf[c]=wdf[c].fillna(wdf.groupby("station_id")[c].transform("median"))

#apply the function in a for loop
for c in ['pH', 'turbidity_NTU',
       'temperature_C', 'dissolved_oxygen_mgL', 'nitrate_mgL', 'lead_ugL']:
  med(c)

#flag high lead and turb
wdf["high_lead"]=wdf["lead_ugL"]>15
wdf["high_turb"]=wdf["turbidity_NTU"]>5


#now extract month and year
wdf["month"]=wdf["sample_time"].dt.to_period("M")

#now extract if that day was weekend
wdf["is_weekend"]=wdf["sample_time"].dt.day_of_week>5

#lets see if it was day-time
wdf["is_daytime"]=wdf["sample_time"].dt.hour.between(9,17)

#now lets merge with stations

df=wdf.merge(sdf, how="left", on="station_id")
#print(df.info())

df["station_id"].unique()
#print(wdf.head())

#Monthly per-station means for: pH, turbidity_NTU, temperature_C, dissolved_oxygen_mgL, nitrate_mgL, lead_ugL, e_coli_CFU_100mL.
for c in ["pH", "turbidity_NTU", "temperature_C", "dissolved_oxygen_mgL", "nitrate_mgL", "lead_ugL", "e_coli_CFU_100mL"]:
  a=df.groupby("month")[c].mean()
  #print(a)

  #Station-month % samples with qc_flag != 'ok'.
#print(df.head())
#print(df.columns)


# Station-month % samples with qc_flag != 'ok'. using. agg for the first time
c=df.groupby(["station_id","month"]).agg(tot=("qc_flag", "count"), bad=("qc_flag", lambda x: (x!="ok").sum()))
c["%"]=c["bad"]/c["tot"]
#print(c)



#Station-month count where flag_high_lead or flag_high_turb is True.
e=df.groupby(["station_id", "month"]).agg(flag_lead=('high_lead', "sum"), high_turb=("high_turb", "sum"))
print(e.head())
v=0

(e["flag_lead"] +e["high_turb"] >0).sum()

#Export a tidy summary kpis_monthly.csv with columns:
#station_id, month, mean_ph, mean_turbidity, mean_temp, mean_do,
# mean_nitrate, mean_lead, mean_ecoli, pct_qc_issues, cnt_flag_high_lead, cnt_flag_high_turb


means = (
    df.groupby(["station_id", "month"], as_index=False)
      .agg(mean_ph=('pH','mean'),
           mean_turbidity=('turbidity_NTU','mean'),
           mean_temp=('temperature_C','mean'),
           mean_do=('dissolved_oxygen_mgL','mean'),
           mean_nitrate=('nitrate_mgL','mean'),
           mean_lead=('lead_ugL','mean'),
           mean_ecoli=('e_coli_CFU_100mL','mean'))
)

# --- 2) Station-month % QC issues
qc = (
    df.groupby(["station_id","month"])
      .agg(total_rows=('qc_flag','count'),
           bad_rows=('qc_flag', lambda x: (x != 'ok').sum()))
      .assign(pct_qc_issues=lambda d: d['bad_rows'] / d['total_rows'] * 100)
      .reset_index()[['station_id','month','pct_qc_issues']]
)

# --- 3) Station-month counts of high_lead / high_turb
flags = (
    df.groupby(["station_id","month"], as_index=False)
      .agg(cnt_flag_high_lead=('high_lead','sum'),
           cnt_flag_high_turb=('high_turb','sum'))
)

# --- 4) Merge everything into one tidy table
kpis = (
    means.merge(qc, on=['station_id','month'], how='left')
         .merge(flags, on=['station_id','month'], how='left')
)

# --- 5) Nice-to-haves: sort, round, and ensure month is a string for CSV
kpis = kpis.sort_values(['station_id','month'])
num_cols_to_round = ['mean_ph','mean_turbidity','mean_temp','mean_do',
                     'mean_nitrate','mean_lead','mean_ecoli','pct_qc_issues']
kpis[num_cols_to_round] = kpis[num_cols_to_round].round(2)
kpis['month'] = kpis['month'].astype(str)  # Period[M] -> 'YYYY-MM'

# --- 6) Export
kpis.to_csv('kpis_monthly.csv', index=False)
print("✅ Saved kpis_monthly.csv with shape:", kpis.shape)
print(kpis.head())




                    flag_lead  high_turb
station_id month                        
s01        2025-01          0          0
           2025-02          1          3
           2025-03          1          2
           2025-04          0          1
s02        2025-01          0          1
✅ Saved kpis_monthly.csv with shape: (32, 12)
  station_id    month  mean_ph  mean_turbidity  mean_temp  mean_do  \
0        s01  2025-01     7.69            3.55      15.49     9.67   
1        s01  2025-02     6.95            4.24      14.59     9.47   
2        s01  2025-03     7.09            4.11      13.03     8.72   
3        s01  2025-04     7.24            3.38      21.51     9.49   
4        s02  2025-01     7.33            5.39       9.08    10.24   

   mean_nitrate  mean_lead  mean_ecoli  pct_qc_issues  cnt_flag_high_lead  \
0          1.62       5.30        20.5          50.00                   0   
1          1.39       6.66        14.0          28.57                   1   
2          1.35